##### Copyright &copy; 2020 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Model Analysis
***An Example of a Key Component of TensorFlow Extended (TFX)***

Note: You can run this example right now in a Jupyter-style notebook, no setup required!  Just click "Run in Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic">
<img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/model_analysis/tfma_basic.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/model_analysis/tfma_basic.ipynb">
<img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
</table></div>



This example colab notebook illustrates how TensorFlow Model Analysis (TFMA) can be used to investigate and visualize the characteristics of a dataset and the performance of a model.  We'll use a model that we trained previously, and now you get to play with the results!

The model we trained was for the [Chicago Taxi Example](https://github.com/tensorflow/model-analysis/tree/master/examples/chicago_taxi), which uses the [Taxi Trips dataset](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) released by the City of Chicago.

Note: This site provides applications using data that has been modified for use from its original source, www.cityofchicago.org, the official website of the City of Chicago. The City of Chicago makes no claims as to the content, accuracy, timeliness, or completeness of any of the data provided at this site. The data provided at this site is subject to change at any time. It is understood that the data provided at this site is being used at one’s own risk.

[Read more](https://cloud.google.com/bigquery/public-data/chicago-taxi) about the dataset in [Google BigQuery](https://cloud.google.com/bigquery/). Explore the full dataset in the [BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips).

Key Point: As a modeler and developer, think about how this data is used and the potential benefits and harm a model's predictions can cause. A model like this could reinforce societal biases and disparities. Is a feature relevant to the problem you want to solve or will it introduce bias? For more information, read about <a target='_blank' href='https://developers.google.com/machine-learning/fairness-overview/'>ML fairness</a>.

Key Point: In order to understand `TFMA` and how it works with Apache Beam, you'll need to know a little bit about Apache Beam itself.  The <a target='_blank' href='https://beam.apache.org/documentation/programming-guide/'>Beam Programming Guide</a> is a great place to start.

The columns in the dataset are:
<table>
<tr><td>pickup_community_area</td><td>fare</td><td>trip_start_month</td></tr>

<tr><td>trip_start_hour</td><td>trip_start_day</td><td>trip_start_timestamp</td></tr>
<tr><td>pickup_latitude</td><td>pickup_longitude</td><td>dropoff_latitude</td></tr>
<tr><td>dropoff_longitude</td><td>trip_miles</td><td>pickup_census_tract</td></tr>
<tr><td>dropoff_census_tract</td><td>payment_type</td><td>company</td></tr>
<tr><td>trip_seconds</td><td>dropoff_community_area</td><td>tips</td></tr>
</table>

## Install Jupyter Extensions
Note: If running in a local Jupyter notebook, then these Jupyter extensions must be installed in the environment before running Jupyter.

```bash
jupyter nbextension enable --py widgetsnbextension
jupyter nbextension install --py --symlink tensorflow_model_analysis
jupyter nbextension enable --py tensorflow_model_analysis
```

## Install TensorFlow Model Analysis (TFMA)

This will pull in all the dependencies, and will take a minute.  Please ignore the warnings.

In [0]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [0]:
print('Installing TensorFlow')
!pip install tensorflow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print('TF version: {}'.format(tf.__version__))

print('Installing Apache Beam')
!pip install -Uq apache_beam
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))

# Install TFMA
# This will pull in all the dependencies, and will take a minute
# Please ignore the warnings
!pip install -q tensorflow-model-analysis

import tensorflow as tf
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

## Load The Files
We'll download a tar file that has everything we need.  That includes:

* Training and evaluation datasets
* Data schema
* Training results as EvalSavedModels

Note: We are downloading with HTTPS from a Google Cloud server.

In [0]:
# Download the tar file from GCP and extract it
import io, os, tempfile
BASE_DIR = tempfile.mkdtemp()
TFMA_DIR = os.path.join(BASE_DIR, 'eval_saved_models-0.15.0')
DATA_DIR = os.path.join(TFMA_DIR, 'data')
OUTPUT_DIR = os.path.join(TFMA_DIR, 'output')
SCHEMA = os.path.join(TFMA_DIR, 'schema.pbtxt')

!wget https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/eval_saved_models-0.15.0.tar
!tar xf eval_saved_models-0.15.0.tar
!mv eval_saved_models-0.15.0 {BASE_DIR}
!rm eval_saved_models-0.15.0.tar

print("Here's what we downloaded:")
!ls -R {TFMA_DIR}

## Parse the Schema

Among the things we downloaded was a schema for our data that was created by [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/).  Let's parse that now so that we can use it with TFMA.

In [0]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

## Use the Schema to Create TFRecords

We need to give TFMA access to our dataset, so let's create a TFRecords file.  We can use our schema to create it, since it gives us the correct type for each feature.

In [0]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if len(line[key]) > 0:
      if feature.type == schema_pb2.FLOAT:
        example.features.feature[key].float_list.value[:] = [float(line[key])]
      elif feature.type == schema_pb2.INT:
        example.features.feature[key].int64_list.value[:] = [int(line[key])]
      elif feature.type == schema_pb2.BYTES:
        example.features.feature[key].bytes_list.value[:] = [line[key].encode('utf8')]
    else:
      if feature.type == schema_pb2.FLOAT:
        example.features.feature[key].float_list.value[:] = []
      elif feature.type == schema_pb2.INT:
        example.features.feature[key].int64_list.value[:] = []
      elif feature.type == schema_pb2.BYTES:
        example.features.feature[key].bytes_list.value[:] = []
  examples.append(example)

TFRecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(TFRecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())
  writer.flush()
  writer.close()

!ls {TFRecord_file}

## Setup and Run TFMA

TFMA supports a number of different model types including TF keras models, models based on generic TF2 signature APIs, as well TF estimator based models. The [get_started](https://www.tensorflow.org/tfx/model_analysis/get_started#model-types-supported) guide has the full list of model types supported and any restrictions. For this example we are going to use an estimator based model that was saved as an [`EvalSavedModel`](https://www.tensorflow.org/tfx/model_analysis/eval_saved_model). 

For our [setup](https://www.tensorflow.org/tfx/model_analysis/setup) we will use the built-in metrics that were saved with the model (see the [metrics](https://www.tensorflow.org/tfx/model_analysis/metrics) guide for how to add additional metrics and plots not saved with the model). This setup also includes a number of slicing specs which are discussed in more detail in the following sections.

After creating a `tfma.EvalConfig` and `tfma.EvalSharedModel` we can then run TFMA using [`tfma.run_model_analysis`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis). This will create an `EvalResult` which we can use later for rendering our metrics and plots.

In [0]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # This assumes an eval saved model. Remove the `signature_name` and add a
    # `label_key` if using a keras or serving model.
    signature_name: "eval"
  }

  ## Post export metric information
  metrics_specs {
    # ... add additional metrics and plots ...
    metrics { class_name: "ConfusionMatrixPlot" }
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_keys: ["trip_start_month"]
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our eval saved model.
eval_model_base_dir_0 = os.path.join(TFMA_DIR, 'run_0', 'eval_model_dir')
eval_model_dir_0 = os.path.join(eval_model_base_dir_0, next(os.walk(eval_model_base_dir_0))[1][0])
eval_shared_model = tfma.default_eval_shared_model(eval_saved_model_path=eval_model_dir_0)

# Run TFMA
eval_result = tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                                      eval_config=eval_config,
                                      data_location=TFRecord_file,
                                      file_format='tfrecords',
                                      output_path='sample_data')

### Rendering Metrics

Now that we've run the evaluation, let's take a look at our visualizations using TFMA. To view metrics you use [`tfma.view.render_slicing_metrics`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_slicing_metrics)

By default the views will display the `Overall` slice. To view a particular slice you can either use the name of the column (by setting `slicing_column`) or provide a [`tfma.slicer.SingleSliceSpec`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SingleSliceSpec). 

The metrics visualization is interactive:

* Click and drag to pan
* Scroll to zoom
* Right click to reset the view

Simply hover over the desired data point to see more details.  Select from four different types of views using the selections at the bottom.

For example, we'll be setting `slicing_column` to look at the `trip_start_hour` feature from our previous `slicing_specs`.

In [0]:
tfma.view.render_slicing_metrics(eval_result, slicing_column='trip_start_hour')

### Slices Overview

The default visualization is the **Slices Overview** when the number of slices is small. It shows the values of metrics for each slice. Since we've selected `trip_start_hour` above, it's showing us metrics like accuracy and AUC for each hour, which allows us to look for issues that are specific to some hours and not others.

In the visualization above:

* Try sorting the feature column, which is our `trip_start_hours` feature, by clicking on the column header
* Try sorting by precision, and **notice that the precision for some of the hours with examples is 0, which may indicate a problem**

The chart also allows us to select and display different metrics in our slices.

* Try selecting different metrics from the "Show" menu
* Try selecting recall in the "Show" menu, and **notice that the recall for some of the hours with examples is 0, which may indicate a problem**

It is also possible to set a threshold to filter out slices with smaller numbers of examples, or "weights".  You can type a minimum number of examples, or use the slider.

### Metrics Histogram

This view also supports a **Metrics Histogram** as an alternative visualization, which is also the default view when the number of slices is large. The results will be divided into buckets and the number of slices / total weights / both can be visualized. Columns can be sorted by clicking on the column header.  Slices with small weights can be filtered out by setting the threshold. Further filtering can be applied by dragging the grey band. To reset the range, double click the band. Filtering can also be used to remove outliers in the visualization and the metrics tables. Click the gear icon to switch to a logarithmic scale instead of a linear scale.

* Try selecting "Metrics Histogram" in the Visualization menu


### More Slices

Our initial `tfma.EvalConfig` created a whole list of `slicing_specs`, which we can visualize by updating slice information passed to `tfma.view.render_slicing_metrics`. Here we'll select the `trip_start_day` slice (days of the week).  **Try changing the `trip_start_day` to `trip_start_month` and running again to examine different slices.**

In [0]:
tfma.view.render_slicing_metrics(eval_result, slicing_column='trip_start_day')

TFMA also supports creating feature crosses to analyze combinations of features. Our original settings created a cross `trip_start_hour` and `trip_start_day`:

In [0]:
tfma.view.render_slicing_metrics(
    eval_result, 
    slicing_spec=tfma.slicer.SingleSliceSpec(
        columns=['trip_start_hour', 'trip_start_day']))

Crossing the two columns creates a lot of combinations!  Let's narrow down our cross to only look at **trips that start at noon**.  Then let's select `accuracy` from the visualization:

In [0]:
tfma.view.render_slicing_metrics(
    eval_result, 
    slicing_spec=tfma.slicer.SingleSliceSpec(
        columns=['trip_start_day'], features=[('trip_start_hour', 12)]))

### Rendering Plots

Any plots that were added to the `tfma.EvalConfig` as post export `metric_specs` can be displayed using [`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot).  

As with metrics, plots can be viewed by slice. Unlike metrics, only a single plot is displayed so the `tfma.slicer.SingleSliceSpec` must be used and it must specify a both a slice feature name and value. If no slice is provided then the plot for the `Overall` slice is used.

In the example below we are displaying the `ConfusionMatrixPlot` that was added for the `trip_start_hour:1` slice.

In [0]:
tfma.view.render_plot(
    eval_result,
    tfma.slicer.SingleSliceSpec(features=[('trip_start_hour', 1)]))

## Tracking Model Performance Over Time

Your training dataset will be used for training your model, and will hopefully be representative of your test dataset and the data that will be sent to your model in production.  However, while the data in inference requests may remain the same as your training data, in many cases it will start to change enough so that the performance of your model will change.

That means that you need to monitor and measure your model's performance on an ongoing basis, so that you can be aware of and react to changes.  Let's take a look at how TFMA can help.

### Measure Performance For New Data
We downloaded the results of three different training runs above, so let's load them now and use TFMA to see how they compare using [`render_time_series`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series).  We can specify particular slices to look at.  Let's compare our training runs for trips that started at noon.

* Select a metric from the dropdown to add the time series graph for that metric
* Close unwanted graphs
* Hover over data points (the ends of line segments in the graph) to get more details

In [0]:
# Note this re-uses the tfma.EvalConfig from the previous setup.

# Run eval on each eval saved model
output_paths = []
for i in range(3):
  # Create a tfma.EvalSharedModel that points at our eval saved model.
  eval_model_base_dir = os.path.join(TFMA_DIR, 'run_%d' % i, 'eval_model_dir')
  eval_model_dir = os.path.join(eval_model_base_dir, next(os.walk(eval_model_base_dir))[1][0])
  eval_shared_model = tfma.default_eval_shared_model(eval_saved_model_path=eval_model_dir)

  output_path = os.path.join(TFMA_DIR, 'output', 'run_%d' % i)
  output_paths.append(output_path)

  # Run TFMA
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=eval_config,
                          data_location=TFRecord_file,
                          file_format='tfrecords',
                          output_path=output_path)

### How does it look today?
First, we'll imagine that we've trained and deployed our model yesterday, and now we want to see how it's doing on the new data coming in today.  The visualization will start by displaying accuracy.  Add AUC and average loss by using the "Add metric series" menu.

Note: In the metric series charts the X axis is the model ID number of the model run that you're examining.  The numbers themselves are not meaningful.

In [0]:
eval_results_from_disk = tfma.load_eval_results(
    output_paths[:2], tfma.constants.MODEL_CENTRIC_MODE)

tfma.view.render_time_series(eval_results_from_disk)

Now we'll imagine that another day has passed and we want to see how it's doing on the new data coming in today, compared to the previous two days. Again add AUC and average loss by using the "Add metric series" menu:

In [0]:
eval_results_from_disk = tfma.load_eval_results(
    output_paths, tfma.constants.MODEL_CENTRIC_MODE)

tfma.view.render_time_series(eval_results_from_disk)